### KKT QP
****std form assumed****

Min f(x) = $x^{T}Qx$ + $c^{T}x$ \
sub to $Ax \leq b$ \
      $x \geq 0$ \
      
where $Q = [q_{ij}]_{nxn}$ - symmertric positive semi definite matrix \
$c,x \in R^{n}$  \
$b \in R^{m}$ \
$A = [a_{ij}]_{mxn}$ \

The kkt conditions are \







In [1]:
import numpy as np

In [2]:
def simplex_solver(A, b, c, MIN=0):
    if MIN:
        c = c * -1
    # A, b, c as per the standard form given 
    n_eq = A.shape[0]
    n_unk = c.shape[0] + n_eq #original variables + slack variables
    n_slack = n_eq #no. of slack variables
    #construct initial tableau
    coef_slack_z = np.zeros(n_slack, float)  #initially the coeff of slack variables in z
    coef_in_z = np.hstack((c, coef_slack_z))  #coeff of basic + non-basic variables in z

    coeff_basic = np.zeros((n_slack, 2), float)

    for i in range(coeff_basic.shape[0]):
        coeff_basic[i][0] = c.shape[0]+1+i
        coeff_basic[i][1] = 0

    ## A and an identity matrix of size n_slack * n_slack

    slack_table = np.identity(n_slack, float)

    tableau = np.hstack((A,slack_table))

    finished = False
    unbounded_flag = False
    itr = 0
    print('Maximization Simplex')
    print('====================')

    while not finished:
        itr += 1
        print('iteration: {}'.format(itr))
        print('+++++++++++++++++++')
        print('tableau:')
        print(tableau)
        print('\n')
        #need to calculate dot product of coeff of basic and each column of tableau

        zj = []
        for column in tableau.T:
                zj.append(np.dot(coeff_basic[:,1], column))

        print('zj :{}'.format(zj))

        #find cj-zj : the relative profits for this round


        cj_zj = [c-z for (c,z) in zip(coef_in_z, zj)]
        print('cj-zj : {}'.format(cj_zj))
        max_profit = max(cj_zj)
        print('max profit for this round: {}'.format(max_profit))
        if max_profit > 0:
            #LP is not finished yet
            entering = np.argmax(cj_zj)
            b_i = np.array([a/b if b!=0 else np.inf for (a,b) in zip (b, tableau[:,entering])], float)
            print('b_i : {}'.format(b_i))
            min_ratio = min(b_i)
            if(min_ratio > 0):
                #here the LP is not unbounded
                leaving = np.argmin(b_i)
                print('entering: {}, leaving: {}'.format(entering,leaving))
                pivot = tableau[leaving][entering]
                print('pivot: {}'.format(pivot))

                #divide the line of pivot using the pivot
                #this is the leaving row. 

                next_round_tableau = np.zeros(tableau.shape, float)

                leaving_var = c.shape[0] + leaving + 1
                print('leaving var : {}'.format(leaving_var))

                # since pivot column variable is entering, change the coefficient and index in coeff_basic_var

                coeff_basic[leaving][1] = coef_in_z[entering]
                coeff_basic[leaving][0] = entering+1
                print('coeff_basic_var after updating: {}'.format(coeff_basic))

                #to construct tableau for next round
                #dividing pivot row with pivot
                next_round_tableau[leaving] = tableau[leaving]/pivot

                #for each index, the tranformation is done to obtain new value
                row = 0
                while row != n_eq:
                    if row != leaving:
                        for col in range(tableau.shape[1]):
                            if col != entering:
                                next_round_tableau[row][col] = (tableau[row][col]-((tableau[row][entering]*tableau[leaving][col])/pivot))
                    row += 1


                #finally, the basic variables will form identity matrix. assigning 1 in those indices
                for i in range(next_round_tableau.shape[0]):
                    next_round_tableau[i][i+c.shape[0]] = 1
                tableau = next_round_tableau
                print('\n\n')
            else:
                #unbounded LP case
                unbounded_flag = True
                print('LP unbounded!')
                finished = True
        else:
            finished = True
            if not unbounded_flag:
                print('\n\nSimplex finished\n')
                print('optimal values \(x values index and slack variables\): \n {}'.format(coeff_basic))
            #optimum = np.dot(c,coeff_basic[:,1])


In [3]:
Q = np.array([[1, -1],[-1, 2]]) #coeff of quadrtic components

In [4]:
c = np.array([1, 2]) #coeff of linear part

In [5]:
#constraints
A = np.array([[1, 2], [1, 3]])
b = np.array([2, 2])

In [6]:
A.shape

(2, 2)

In [7]:
c.shape

(2,)

In [8]:
#other identity and zero matrices

In [9]:
In = np.identity(c.shape[0])
I = np.identity(A.shape[0])
zero = np.zeros((A.shape[0], A.shape[0]))
Azero = np.zeros((A.shape[0],A.shape[0]))
Inzero = np.zeros((A.shape[0], A.shape[0]))


In [10]:
In

array([[1., 0.],
       [0., 1.]])

In [11]:
I

array([[1., 0.],
       [0., 1.]])

In [12]:
zero

array([[0., 0.],
       [0., 0.]])

In [13]:
Azero

array([[0., 0.],
       [0., 0.]])

In [14]:
Inzero

array([[0., 0.],
       [0., 0.]])

In [15]:
#solve Lx = v
L = [[2*Q, A.T, -1*In, zero], [A, Azero, Inzero, I]]


In [16]:
L = 2*Q

In [17]:
L

array([[ 2, -2],
       [-2,  4]])

In [18]:
A.T

array([[1, 1],
       [2, 3]])

In [19]:
L1 = np.hstack((2*Q, A.T, -1*In, zero.T))

In [20]:
L2 = np.hstack((A, Azero, Inzero, I))

In [21]:
L1

array([[ 2., -2.,  1.,  1., -1., -0.,  0.,  0.],
       [-2.,  4.,  2.,  3., -0., -1.,  0.,  0.]])

In [22]:
L2

array([[1., 2., 0., 0., 0., 0., 1., 0.],
       [1., 3., 0., 0., 0., 0., 0., 1.]])

In [23]:
L = np.vstack((L1, L2))

In [24]:
v = np.hstack((-1*c, b))

In [25]:
v

array([-1, -2,  2,  2])

In [26]:
L


array([[ 2., -2.,  1.,  1., -1., -0.,  0.,  0.],
       [-2.,  4.,  2.,  3., -0., -1.,  0.,  0.],
       [ 1.,  2.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  3.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [27]:
##these equations form the constraints. there are x, lambda, mu, s, and artifical variables A1,A2,A3 and A4
## Objective is to minimize A1+A2+A3+A4

In [28]:
C = np.ones(L.shape[0])

In [29]:
C

array([1., 1., 1., 1.])

#this now forms a simplex as 

min $C^{T}X$
ST  Lx <= b
and x>=0



In [30]:
simplex_solver(L,v,C,1)

Maximization Simplex
iteration: 1
+++++++++++++++++++
tableau:
[[ 2. -2.  1.  1. -1. -0.  0.  0.  1.  0.  0.  0.]
 [-2.  4.  2.  3. -0. -1.  0.  0.  0.  1.  0.  0.]
 [ 1.  2.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 1.  3.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.]]


zj :[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
cj-zj : [-1.0, -1.0, -1.0, -1.0, 0.0, 0.0, 0.0, 0.0]
max profit for this round: 0.0


Simplex finished

optimal values \(x values index and slack variables\): 
 [[5. 0.]
 [6. 0.]
 [7. 0.]
 [8. 0.]]
